In [ ]:
def ghard(col, mode = 'sequential_fixed', fit = 'fit_transform', params = None,
          max_target = 2, max_gap_ratio = 100, log_stretch = 10):
    '''
    mode : str, default = 'sequential_fixed'
        'sequential_fixed' : recursive natural log transformations of quartiles with outliers
        'max' : scale outliers to max_target times IQR by compressing the first
            and fourth quartiles
        'gap_logger': order data, calculate all the gaps between unique values in the data,
            get median gap, get the max gap, set the minimum gap to one, and raise gaps
            to a polynomial power that scale the max gap to max_gap_ratio times the median gap
            (default 100). Rescale data back to 0 to 1
        'simple_log': for universally positive integers, takes the natural log.
        'dynamic_log': determines if the outliers are at the bottom or top of the range, or both.
            For very small/negative outliers shifts the max to -1, scales the min to -log_stretch times
                the max-median gap, takes the -log(-x) for each value. Max capped at 0.
            For very large outliers shifts the min to 1, scales the max to log_stretch times
                the min-median gap, takes the log(x) for each value. Min capped at 0.
            For outliers on both ends, shifts the median to 1, scales the max to log_stretch times
                the inter-quartile range multiple of the max, takes the log(x) for each value
                above the median. shifts the value of the min to log_stretch times
                the inter-quartile range multiple of the min + 2, then subtracts 2 from all values
                below the median and takes the - log(-x) of thos values (split logging)
    fit: string ['fit', 'transform', 'fit_transform'], default = 'fit_transform'
        Like with min max scaler, fit returns params, used for consistent transformation.
    params: dictionary, default = None
        Scaling parameter dictionary generated by fitting.
    max_target: float, default = 2
        What multiple of the inter quartile range should be the goal of the compression?
    max_gap_ratio: int, default = 100.
        For gap_logger, what is the ratio of the largest gap to the median.
    log_stretch: int, default = 10.
        For the dynamic log mode, determines the level of log compression for outliers.
    '''

    # Perform fit-transform as a sequential call to fit then transform
    if fit == 'fit_transform':
        params = ghard(col, mode = mode, fit = 'fit', max_target = max_target, max_gap_ratio = max_gap_ratio)
        return ghard(col, mode = mode, fit = 'transform', params = params,
                     max_target = max_target, max_gap_ratio = max_gap_ratio)

    elif fit == 'fit':
        # collect parameters to return in a dictionary
        params = {}

        # confirm data is formatted as a pandas series
        if isinstance(col, pd.core.frame.DataFrame):
            if col.shape[1] != 1:
                raise ValueError('Unsupported datatype: multi-column DataFrame')
            col_name = col.columns[0]
            col_raw = col[[col_name]].copy().iloc[:, 0]
        elif isinstance(col, pd.core.series.Series):
            col_raw = col.copy()
        elif isinstance(col, np.ndarray):
            # TODO: convert all column names to col???
            col_raw = pd.Series(col, name = 'col')
        else:
            raise ValueError('Unsupported datatype: must be a pandas series, dataframe or numpy array')

        # ensure column is not single value or binary
        if (col_raw.nunique() < 3):
            params['transform'] = False
            return params

        # get interquartile range (IQR)
        iq1, iq3 = non_zero_iqr(col_raw, pct = 0.25)

        # make sure column has minimum variance
        if (iq1 == 0) and (iq3 == 0):
            params['transform'] = False
            return params

        # copy and transform data so IQR is from -1 to 1
        col_ = 2 * (col_raw - iq1) / (iq3 - iq1) - 1

        # determine the largest outlier
        max_ = col_.max()
        min_ = col_.min()
        max_outlier_abs = max([max_, -min_])

        # determine if largest outlier requires compression
        if max_outlier_abs < (2 * max_target + 1):
            params['transform'] = False
            return params

        params['mode'] = mode
        params['transform'] = True
        params['iq1'] = [iq1]
        params['iq3'] = [iq3]
        params['col_max'] = [max_]
        params['col_min'] = [min_]
        params['max_outlier_abs'] = max_outlier_abs
        params['max_target'] =  max_target

        if mode == 'max':
            # determine the log base that scales the largest outlier to the
            # max_target
            log_base = max_outlier_abs**(1/(2 * max_target))
            params['log_base'] = log_base

            return params

        elif mode == 'sequential_fixed':
            while (col_.max() > (max_target * 2 + 1)) or (col_.min() < - (max_target * 2 + 1)):
                if col_.max() > (max_target * 2 + 1):
                    if col_.min() < -np.log(max_target * 2 + 1):
                        col_ = col_.apply(lambda x: log_tail(x, side = 'both'))
                    else:
                        col_ = col_.apply(lambda x: log_tail(x, side = 'right'))
                elif col_.min() < - (max_target * 2 + 1):
                    if col_.max() > np.log(max_target * 2 + 1):
                        col_ = col_.apply(lambda x: log_tail(x, side = 'both'))
                    else:
                        col_ = col_.apply(lambda x: log_tail(x, side = 'left'))
                else:
                    break

                # recompute new min and max
                max_ = col_.max()
                min_ = col_.min()

                params['col_max'].append(max_)
                params['col_min'].append(min_)


            return params

        elif mode == 'gap_logger':
            # what maw percent of unique values allowed in one mesh section
            MAX_FRACTION = .05

            # how many unique values needed in the data
            MIN_UNIQUE = 3

            # if the gap ratio is fixed, how much larger should the largest be
            # from the median
            max_gap_ratio = 1000

            # if the target ratio between the largest gap and median one should
            # scale with the size of the current gap
            dynamic_gap_ratio = True

            if col_raw.nunique() <= MIN_UNIQUE:
                params['transform'] = False
                return params

            # increments that evenly divide 2
            mesh_grids = [.0001, .0002, .0004, .0005, .0008, .001, .00125, .0016,
                          .002, .0025, .004, .005, .00625, .008, .01, .0125, .02,
                          .025, .03125, .04, .05, .0625, .1, .125, .25]

            # get original column min, max and name
            raw_min = col.min()
            raw_max = col.max()
            params['raw_max'] = raw_max
            params['raw_min'] = raw_min
            col_name = str(col_raw.name)

            # fit the data to a min -1, max of 1
            col_ = 2 * (col_raw - raw_min) / (raw_max - raw_min) - 1

            # put unique values in a sorted numpy array and dataframe
            unique_vals = np.sort(col_.dropna().unique())
            unique_df = pd.DataFrame(unique_vals, columns = [col_name])

            # calculate the max number of values allowable in a window
            window = int(len(unique_df) * MAX_FRACTION) + 1

            # look at spacing of the unique values in df
            unique_df['diff'] = unique_df[col_name].diff(window)
            min_win = unique_df['diff'].min()

            # assign a mesh that has enough detail to capture features, but large
            # enough to not overfit the fit column
            mesh_grid = max([x for x in mesh_grids if x < min_win] + [0.00005])

            # get all the mesh grid intervals, put in a dataframe tag id =1
            steps = round(2/mesh_grid)
            grid = [x/steps - 1 for x in range(2 * steps + 1)]
            grid_df = pd.DataFrame(grid, columns = [col_name])
            grid_df['id'] = 1

            # get the uniqe values in a df with id = 2
            unique_df = unique_df.drop(columns= ['diff'])
            unique_df['id'] = 2

            # merge mesh and unique real data points, sort together
            unique_mesh = pd.concat([unique_df, grid_df], axis  = 0)
            unique_mesh = unique_mesh.sort_values(by = [col_name, 'id'], ascending = [1,0])

            # make a column of ids from the row above
            unique_mesh['shift_id'] = unique_mesh['id'].shift(1).fillna(2)

            # compute type. seqential mesh points have type 0 and can be deleted
            unique_mesh['type'] = unique_mesh['id'] * unique_mesh['id'] - \
                                  unique_mesh['shift_id']
            unique_mesh = unique_mesh.loc[unique_mesh['type'] != 0]

            # separate mesh points from actual data for computing the mesh grid
            unique_mesh = unique_mesh.sort_values(by = ['id', col_name], ascending = [0,1])

            # get the gaps between the unique values
            unique_mesh['gaps'] = unique_mesh[col_name].diff()

            # set the mesh gaps to NAN so won't affect gap stats
            unique_mesh.loc[unique_mesh['id'] == 1, 'gaps'] = np.nan

            # find min, max and media gap size
            median_gap = unique_mesh['gaps'].median()
            min_gap = unique_mesh['gaps'].min()
            max_gap = unique_mesh['gaps'].max()

            # how big is the largest gap between data points compared to the mean
            current_gap_ratio = max_gap/ median_gap

            # THIS NEEDS TUNING!!! JUST ARBITRARY BASED ON A FEW COLUMNS
            if dynamic_gap_ratio:
                max_gap_ratio = 10 * current_gap_ratio ** 0.5

            # protect against zero division, get the exponent that scales median
            # gap and maximum gap to the desired ratio
            if max_gap == median_gap:
                power = 1
            else:
                power = np.log(max_gap_ratio) / (np.log(max_gap) - np.log(median_gap))
            params['power'] = power

            # normalize the gap values with power scaling
            def power_scale_gaps(x):
                if np.isnan(x):
                    return x
                return (x / min_gap)**power

            # scale gaps to the defined ratio of max to median
            unique_mesh['power_scale_gaps'] = unique_mesh['gaps'].apply(power_scale_gaps)

            # fill the NaN created by the shift and cumulatively sum the scaled gaps
            unique_mesh['power_scale_gaps_sum'] = unique_mesh['power_scale_gaps'].fillna(0)
            unique_mesh['power_scale_gaps_sum'] = unique_mesh['power_scale_gaps_sum'].cumsum()

            # bring the mesh points back in sorted order with the column values
            unique_mesh = unique_mesh.sort_values(by = [col_name, 'id'], ascending = [1,0])

            # due to the sort, every mesh point has a preceeding actual value
            # use shift to get the scaled position of that point in the same row
            unique_mesh['power_scale_gaps_sum_shift'] = unique_mesh['power_scale_gaps_sum'].shift(1)

            # get the gap between the grid point and the preceeding point
            unique_mesh[col_name + '_diff'] = unique_mesh[col_name].diff(1)

            # copy just the mesh grid points into a new dataframe
            # option to clean up with: .drop(columns = drop_cols)
            # drop_cols = ['id', 'shift_id', 'type', 'gaps', 'power_scale_gaps', 'power_scale_gaps_sum']
            unique_mesh_grid = unique_mesh.loc[unique_mesh['id'] == 1].copy()

            # scale the mesh points just like an actual data point using the gap
            # from the previous data point
            unique_mesh_grid['power_scale_diff'] = unique_mesh_grid[
                                    col_name + '_diff'].apply(power_scale_gaps)

            # add the scaled position from the actual data to the scaled gap
            unique_mesh_grid['final_shift'] = unique_mesh_grid['power_scale_gaps_sum_shift'] + \
                                  unique_mesh_grid['power_scale_diff']

            # get the max value of the final dilated dataframe (min will be zero)
            max_final = unique_mesh_grid['final_shift'].max()

            # divide by range and shift so points are on a -1 to 1 scale
            unique_mesh_grid['final_shift_scaled'] = 2 * unique_mesh_grid['final_shift'] / max_final - 1

            # save the mesh grid points and their transformed positions
            params['grid_shift'] = unique_mesh_grid[[col_name, 'final_shift_scaled']].values

            return params

        # set up for log transformation
        elif mode in ['simple_log', 'dynamic_log']:
            # get original column min, max and name
            params['raw_max'] = col_raw.max()
            params['raw_min'] = col_raw.min()
            params['raw_med'] = col_raw.median()

            return params

        else:
            print('Error. No transformation performed')
            params['transform'] = False
            return params

    # transform column
    elif fit == 'transform':

        # if the fit determined no outliers or improper column
        # like boolean, simply return the original column
        if (not params['transform']) or params['no_outliers']:
            return col

        # confirm data is formatted as a pandas series
        if isinstance(col, pd.core.frame.DataFrame):
            if col.shape[1] != 1:
                raise ValueError('Unsupported datatype: multi-column DataFrame')
            col_name = col.columns[0]
            col_raw = col[[col_name]].copy().iloc[:, 0]
        elif isinstance(col, pd.core.series.Series):
            col_raw = col.copy()
        elif isinstance(col, np.ndarray):
            # TODO: convert all column names to col???
            col_raw = pd.Series(col, name = 'col')
        else:
            raise ValueError('Unsupported datatype: must be a pandas series, dataframe or numpy array')

        # get interquartile range (IQR)
        iq1, iq3 = (params['iq1'][0], params['iq3'][0])

        # transform the raw column
        col_ = 2 * (col_raw - iq1) / (iq3 - iq1) - 1

        # force parameters to match the fit parameters
        mode = params['mode']
        max_ = params['col_max'][0]
        min_ = params['col_min'][0]
        max_outlier_abs = params['max_outlier_abs']
        max_target = params['max_target']

        if mode == 'max':
            # determine the log base that scales the largest outlier to the
            # max_target
            log_base = params['log_base']

            col_ = col_.apply(lambda x: log_tail(x, side = 'both', log_base = log_base))

            return col_

        elif mode == 'sequential_fixed':
            count = 1
            while (max_ > (max_target * 2 + 1)) or (min_ < - (max_target * 2 + 1)):
                if max_ > (max_target * 2 + 1):
                    if col_.min() < -np.log(max_target * 2 + 1):
                        col_ = col_.apply(lambda x: log_tail(x, side = 'both'))
                    else:
                        col_ = col_.apply(lambda x: log_tail(x, side = 'right'))
                elif min_ < - (max_target * 2 + 1):
                    if max_ > np.log(max_target * 2 + 1):
                        col_ = col_.apply(lambda x: log_tail(x, side = 'both'))
                    else:
                        col_ = col_.apply(lambda x: log_tail(x, side = 'left'))
                else:
                    break

                # set the min and max from the fit
                max_ = params['col_max'][count]
                min_ = params['col_min'][count]

                count += 1

            return col_

        elif mode == 'gap_logger':

            raw_max = params['raw_max']
            raw_min = params['raw_min']
            col_name = col_raw.name

            # fit the data to a min -1, max of 1
            col_ = 2 * (col_raw - raw_min) / (raw_max - raw_min) - 1

            # put unique values in a sorted numpy array and dataframe
            unique_vals = np.sort(col_.dropna().unique())
            unique_df = pd.DataFrame(unique_vals, columns = ['values'])
            unique_df['id'] = 2

            # get all the mesh grid intervals, put in a dataframe tag id =1
            grid = params['grid_shift']
            grid_df = pd.DataFrame(grid, columns = ['values', 'transformed'])
            grid_df['id'] = 1
            grid_df['next_mesh_val'] = grid_df['values'].shift(-1).fillna(1)
            grid_df['next_trans_val'] = grid_df['transformed'].shift(-1).fillna(1)

            # merge mesh and unique real data points, sort together
            unique_mesh = pd.concat([unique_df, grid_df], axis  = 0)
            unique_mesh = unique_mesh.sort_values(by = ['values', 'id'],
                                    ascending = [1,1]).reset_index(drop = True)

            # loop over the data to shift each value to the transformed mesh
            iter_ = unique_mesh.copy()
            for row in iter_.itertuples():
                if row.values < -1:
                    # log transform negative outliers
                    unique_mesh.loc[row.Index, 'transformed'] = -np.log(-row.values) - 1
                elif row.values > 1:
                    # log transform positive outliers
                    unique_mesh.loc[row.Index, 'transformed'] = np.log(row.values) + 1
                else:
                    # if this a mesh value, update the moving windows
                    if not np.isnan(row.transformed):
                        curr_val = row.values
                        next_val = row.next_mesh_val
                        curr_trans = row.transformed
                        next_trans = row.next_trans_val
                    else:
                        # for the last value (mesh = 1)
                        if (next_val - curr_val) == 0:
                            unique_mesh.loc[row.Index, 'transformed'] = curr_val

                        # take the percent of the mesh window and scale the data
                        # to the proportional value in the transformed mesh
                        else:
                            pct_val_gap = (row.values - curr_val) / (next_val - curr_val)
                            trans_loc = pct_val_gap * (next_trans - curr_trans) + curr_trans
                            unique_mesh.loc[row.Index, 'transformed'] = trans_loc

            # map each value to the new transformed position
            transformation = pd.Series(unique_mesh['transformed'].values,
                                       index=unique_mesh['values'].values).to_dict()

            #transform the column and shift to 0-1 scale
            col_trans = col_.map(transformation)
            col_trans = (col_trans + 1) / 2

            return col_trans

        elif mode == 'simple_log':
            # get fit columns max and min
            raw_max = params['raw_max']
            raw_min = params['raw_min']
            max_outlier_abs = params['max_outlier_abs']

            # fit the data to a min of 1
            if raw_min < 1:
                shift = 1 - raw_min
            else:
                shift = 0
            col_ = col_raw.values + shift

            # protect against infinite logs, cap at 1
            col_ = np.where(col_ < 1, 1, col_)

            return pd.Series(np.log(col_), name = col_raw.name)

        elif mode == 'dynamic_log':
            # get fit columns max, min, and median
            raw_max = params['raw_max']
            raw_min = params['raw_min']
            raw_med = params['raw_med']

            # determine if median is closer to min or max
            med_loc = (raw_med - raw_min) / (raw_max - raw_min)

            # TODO: make this a hyperparamter?
            median_imbalance = 0.1

            if med_loc < median_imbalance: # large outlier only
                # stretch the data from 1 to a positive value determined by
                # how skewed the data is
                scale = 1 / max(med_loc, .00001) * log_stretch
                col_ = scale * (col_raw.values - raw_min) / (raw_max - raw_min) + 1

                # protect against infinite logs, cap at 1
                col_ = np.where(col_ < 1, 1, col_)

                return pd.Series(np.log(col_), name = col_raw.name)

            elif med_loc > (1 - median_imbalance): # small outlier only
                # stretch the data from -1 to a negative value determined by
                # how skewed the data is
                scale = 1 / max((1 - med_loc), .00001) * log_stretch
                col_ = scale * (col_raw.values - raw_min) / (raw_max - raw_min) - scale - 1

                # protect against infinite logs, cap at 1
                col_ = np.where(col_ > -1, -1, col_)

                return pd.Series(-np.log(-col_), name = col_raw.name)

            else: # outliers on both extremes
                # get how many multiples of the IQR the outliers were
                max_iqr = params['col_max'][0]
                min_iqr = params['col_min'][0]

                # stretch the data proportional to how many iqr intervals each outlier was
                # and shift it so median is at 1
                scale = (max_iqr - min_iqr) * log_stretch
                shift = scale * (raw_med - raw_min) / (raw_max - raw_min)
                col_ = scale * (col_raw - raw_min) / (raw_max - raw_min) - shift

                # take the log of the data >= 1 and -log(-x) of all other data,
                # shifted so all values are < -1
                col_ = np.where(col_ < 1, -np.log(-(col_ - 2)), np.log(col_))

                return pd.Series(col_, name = col_raw.name)

        else:
            print('Something went wrong! No transformation performed.')
            return col

    else:
        print('Something went wrong! No transformation performed.')
        return None

def log_tail(x, side = 'both', log_base = False):
    '''log compresses tail distributions after scaling IQR to -1 to 1.
    side: ['both', 'left', 'right', 'right_only', 'left_only']
        Which tail to compress.
    log_base is used to scale the compression to the outlier max value
        in 'max' style compression
    '''
    if not isinstance(x, (int, float,)):
        raise ValueError('Non-numerical value passed')

    # for sequential natural log
    if not log_base:
        if (x >= -1) and (x <= 1):
            return x
        elif x > 1:
            if side in ['both', 'right']:
                return np.log(x) + 1
            else:
                return x
        elif x < 1:
            if side in ['both', 'left']:
                return - np.log(-x) - 1
            else:
                return x
        else:
            return np.nan

    # for symmetric max
    else:
        if (x >= -1) and (x <= 1):
            return x
        elif x > 1:
            return np.log(x) / np.log(log_base) + 1
        elif x < 1:
            return - np.log(-x) / np.log(log_base) - 1
        else:
            return np.nan

def non_zero_iqr(col, pct = 0.25):
    '''Caluclates the inter quartile range (IQR). If zero, expands the search for a
    non-zero interval between percentiles symmetric around the median and returns
    those values. If the 0.1 percentile and 99.9 percentile values are the same,
    returns (0, 0) to indicate that minimal variance is not present in the data.'''
    # get IQR, default is 25th and 75th percentile
    iq1 = col.quantile([pct]).iloc[0]
    iq3 = col.quantile([1 - pct]).iloc[0]

    # if IQR is zero, expand range
    if iq3 - iq1 == 0:
        # TEMP DELETE
        # print('iq1 = IQ3')
        for i in [x/1000 for x in range(int((1 - pct) * 1000), 1001)]:
            iq1 = col.quantile([1 - i]).iloc[0]
            iq3 = col.quantile([i]).iloc[0]
            # TEMP DELETE
            # print(f'i {i} iq1 {iq1} iq3 {iq3}')
            if iq3 - iq1 != 0:
                # TEMP DELETE
                # print('break')
                break
        # TEMP DELETE
        # print(f'OUT OF LOOP i = {i} i == 1 {i == 1}')

        # if > 99.9% of values the same
        if i == 1:
            return (0, 0)

    return (iq1, iq3)

def non_zero_iqr_percentile(col, pct = 0.25):
    '''Caluclates the percentile in the data for the first non-zero
    inter quartile range (IQR). If the 0.1 percentile and 99.9 percentile values are the same,
    returns 0 to indicate that minimal variance is not present in the data.'''
    # get IQR, default is 25th and 75th percentile
    iq1 = col.quantile([pct]).iloc[0]
    iq3 = col.quantile([1 - pct]).iloc[0]

    # if IQR is zero, expand range
    if iq3 - iq1 == 0:
        for i in [x/1000 for x in range(int((1 - pct) * 1000), 1001)]:
            iq1 = col.quantile([1 - i]).iloc[0]
            iq3 = col.quantile([i]).iloc[0]
            if iq3 - iq1 != 0:
                break
        # if > 99.9% of values the same
        if i == 1:
            return 0
        else:
            return i

    return pct